In [1]:
from mip import *
from backend import *
from itertools import product
import time

In [2]:
network_name = 'usa_99'

# The network
g = nx.read_gml(f'networks/{network_name}.gml', label="id")
L = len(g.edges)
links = range(L)


# The cut SRLGs
with open (f'min_cut_SRLGs/{network_name}', 'rb') as fp:
    cut_srlgs = pickle.load(fp)
S = len(cut_srlgs)


# All SRLG
all_srlgs, _ = get_SRLGs('PSRLGs/usa_99_complete_it6.xml')


# The matrix of the intensity values, dimensions: [L,P,M] (link, position, magnitude)
intensity = np.load(f'intensities/{network_name}_ds23.npy')


# The matrix of earthquake probabilities, dimensions: [P,M] (position, magnitude)
prob_matrix = pd.read_csv('earthquake_probabilities/usa_ds23.csv').drop(['Lat', 'Long'], axis=1).to_numpy()
P, M = prob_matrix.shape
epicenters = range(P)
magnitudes = range(M)


In [ ]:
# Compressing the problem, to 1 SLRG and the minimum number of earthquakes
cut_srlgs = cut_srlgs[:30]
S = len(cut_srlgs)
print(cut_srlgs)

column_mask = np.full(intensity.shape[2], fill_value=False, dtype=bool)
row_mask = np.full(intensity.shape[1], fill_value=False, dtype=bool)

for srlg in cut_srlgs:
    srlg_mask = np.full(intensity.shape[1:], fill_value=True, dtype=bool)
    for link in srlg:
        idx = list(g.edges).index(link)
        print(idx, link, g.edges[link]['length'])
        link_mask = intensity[idx]>6
        srlg_mask &= link_mask
    column_mask |= srlg_mask.any(axis=0)
    row_mask |= srlg_mask.any(axis=1)

intensity = intensity[:,:,column_mask][:,row_mask]
prob_matrix = prob_matrix[:,column_mask][row_mask]

P, M = prob_matrix.shape
epicenters = range(P)
magnitudes = range(M)

print(f'The shape of the intensity matrix: {intensity.shape}')

In [19]:
srlg = cut_srlgs[2]
edges = list(g.edges)
link_mask = np.full(intensity.shape[1:], fill_value=True, dtype=bool)
for l in srlg:
    l_idx = edges.index(l)
    link_mask &= intensity[l_idx] > H[l_idx]


3718

In [ ]:
upgraded_edges = {}
spine_bonus = 0
Ts = [0.01, 0.005, 0.001, 0.0005]

data_totals = {
    'TFA': [],
    'Cut SRLGs': [],
    'Cost H1': [],
    'Cost H2': [],
    'Cost ILP': [],
    'Runtime H1': [],
    'Runtime H2': [],
    'Runtime ILP': [],
}

data_tfa = {
    'Links': list(links).copy(),
    'Link Lengths': np.array([g.edges[e]['length'] for e in g.edges]),
    'H0': np.array([6+spine_bonus*g.edges[e]['onspine'] for e in g.edges]),
    'Delta H (Heuristic 1)': [],
    'Delta H (Heuristic 2)': [],
    'Delta H (ILP)': np.array([np.nan for e in g.edges]),
    'H (Heuristic 1)': [],
    'H (Heuristic 2)': [],
    'H (ILP)': np.array([np.nan for e in g.edges]),
}

for idx,TFA in enumerate(np.concatenate((np.arange(0.01, 0.001, -0.001), np.arange(0.001, 0.0004, -0.0001)))):
    print('\n================')
    print(f'T: {TFA:.4f}')
    data_totals['TFA'].append(TFA)
    
    data_totals['Cut SRLGs'].append(len(cut_srlgs))
    
    H = np.array([6+spine_bonus*g.edges[e]['onspine'] for e in g.edges])
    # Heuristic 1
    start = time.perf_counter()
    H1, cost_1 = heuristic(1, cut_srlgs, g, intensity, H, prob_matrix, TFA)
    runtime_1 = time.perf_counter() - start
    data_totals['Cost H1'].append(cost_1)
    data_totals['Runtime H1'].append(runtime_1)
    data_tfa['Delta H (Heuristic 1)'] = H1 - data_tfa['H0']
    data_tfa['H (Heuristic 1)'] = np.copy(H1)
    
    for T in Ts:
        active_srlgs = [srlg for srlg in all_srlgs if get_SRLG_probability(srlg, g, intensity, H1, prob_matrix)>T]
        write_networkx_to_srg(f'results/{network_name}/{network_name}_TFA{TFA:.4f}_T{T}_H1_SB{spine_bonus}.srg', g, active_srlgs)
    
    
    H = np.array([6+spine_bonus*g.edges[e]['onspine'] for e in g.edges])
    # Heuristic 2
    start = time.perf_counter()
    H2, cost_2 = heuristic(2, cut_srlgs, g, intensity, H, prob_matrix, TFA)
    runtime_2 = time.perf_counter() - start
    data_totals['Cost H2'].append(cost_2)
    data_totals['Runtime H2'].append(runtime_2)
    data_tfa['Delta H (Heuristic 2)'] = H2 - data_tfa['H0']
    data_tfa['H (Heuristic 2)'] = np.copy(H2)
    
    for T in Ts:
        active_srlgs = [srlg for srlg in all_srlgs if get_SRLG_probability(srlg, g, intensity, H2, prob_matrix)>T]
        write_networkx_to_srg(f'results/{network_name}/{network_name}_TFA{TFA:.4f}_T{T}_H2_SB{spine_bonus}.srg', g, active_srlgs)
    
    data_totals['Cost ILP'].append(np.nan)
    data_totals['Runtime ILP'].append(np.nan)
    
    pd.DataFrame(data_tfa).to_csv(f'results/{network_name}/upgrade_{network_name}_TFA{TFA:.4f}_SB{spine_bonus}.csv', index=False, float_format='%.4f')
    
#     H = np.array([6+spine_bonus*g.edges[e]['onspine'] for e in g.edges])
#     upgrade_1, upgrade_2 = [],[]
#     for l,e in enumerate(g.edges):
#         if H1[l] > H[l]:
#             upgrade_1.append((e,int(H1[l]-6)))
#         if H2[l] > H[l]:
#             upgrade_2.append((e,int(H2[l]-6)))
    
#     upgraded_edges[idx] = {}
#     upgraded_edges[idx]['Upgrade H1'] = upgrade_1
#     upgraded_edges[idx]['Upgrade H2'] = upgrade_2

    print('\n================')

pd.DataFrame(data_totals).to_csv(f'results/{network_name}/comparison_{network_name}_SB{spine_bonus}.csv', index=False, float_format='%.4f')

In [70]:
pd.DataFrame(data).to_csv('results/Heuristic_comparison_usa_99_nospine.csv', index=False, float_format='%.4f')
with open('results/Heuristic_upgraded_edges_usa_99_nospine', 'wb') as fp:
    pickle.dump(upgraded_edges, fp)

In [73]:
# ILP

start = time.perf_counter()

#Model
model = Model(sense=MINIMIZE, solver_name=GRB)

#Variables
deltaH = [model.add_var(var_type=INTEGER, lb=0, ub=3) for l,_ in enumerate(g.edges)]
Z = [[[model.add_var(var_type=BINARY) for k in magnitudes] for j in epicenters] for i in range(S)]
Y = [[[model.add_var(var_type=BINARY) for k in magnitudes] for j in epicenters] for i in links]
W = [[model.add_var(var_type=BINARY) for k in magnitudes] for j in epicenters]
print("%.1f s:\tVáltozók létrehozva..."%(time.perf_counter()-start))


#Objective Function
model.objective = xsum( g.edges[link_id]['length'] * deltaH[link_idx] for link_idx,link_id in enumerate(g.edges) )
print("%.1f s:\tCélfüggvény létrehozva..."%(time.perf_counter()-start))


#Constraint 1
for l,p,m in product(*[links, epicenters, magnitudes]):
    model.add_constr( Y[l][p][m] >= 1 - ((Hnull + deltaH[l]) / intensity[l,p,m]) )
print("%.1f s:\tElső egyenlet létrehozva..."%(time.perf_counter()-start))


#Constraint 2
for (c,s), p, m in product(*[enumerate(cut_srlgs), epicenters, magnitudes]):
    model.add_constr( Z[c][p][m] >= (xsum(Y[list(g.edges).index(linkID)][p][m] for linkID in s) - len(s) + 1) )
print("%.1f s:\tMásodik egyenlet létrehozva..."%(time.perf_counter()-start))

# #Constraint 3
# for c,_ in enumerate(cut_srlgs):
#     model.add_constr(xsum( Z[c][p][m] * prob_matrix[p,m] for p,m in product(epicenters,magnitudes) ) <= T, "c3_"+str(c))
# print("%.1f s:\tHarmadik egyenlet létrehozva..."%(time.perf_counter()-start))

for (c,s), p, m in product(*[enumerate(cut_srlgs), epicenters, magnitudes]):
    model.add_constr( W[p][m] >= Z[c][p][m] )

#Constraint 3
model.add_constr(xsum( W[p][m] * prob_matrix[p,m] for p,m in product(epicenters,magnitudes) ) <= T )
print("%.1f s:\tHarmadik egyenlet létrehozva..."%(time.perf_counter()-start))

#Start optimization
model.optimize()#max_seconds=
print("%.1f s:\tMegoldás megtalálva..."%(time.perf_counter()-start))

57.5 s:	Változók létrehozva...
58.6 s:	Célfüggvény létrehozva...
341.5 s:	Első egyenlet létrehozva...
631.1 s:	Második egyenlet létrehozva...
651.9 s:	Harmadik egyenlet létrehozva...
685.5 s:	Megoldás megtalálva...


In [ ]:
# The probability of occurance of the SRLGs
for c,s in enumerate(cut_srlgs):
    print(s, sum(Z[c][p][m].x * prob_matrix[p,m] for p,m in product(epicenters,magnitudes) ))

In [74]:
# The result
selected = [(deltaH[l].x,e, g.edges[e]['length']) for l,e in enumerate(g.edges) if deltaH[l].x >= 0.5]
print(*selected, sep='\n')
#print([dH.x for dH in deltaH])

(3.0, (22, 24, 0), 379.1998052904187)
(2.0, (23, 24, 0), 544.9774670804052)
(1.0, (23, 25, 0), 1091.9747459465025)


In [7]:
# Add additional constraint
# It does not deletes Constraint 3 but completes it
for c,s in enumerate(cut_srlgs):
    model.add_constr(xsum( Z[c][p][m] * prob_matrix[p,m] for p,m in product(epicenters,magnitudes) ) <= 0.00001, "c3_"+str(c))

In [8]:
# Start optimization
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

# ---------------------------------------------------------------

In [7]:
def ff(a,b):
    return a*2 + b**2

In [14]:
#koltseg =  [11, 3,3, 6, 1, 9, 16,78]
koltseg = 8
fejlesztes = [17,6,4,61,16,42,156, 7]

y = [1,1,0,1,0,1,0,0]
z = [0,1,1,1,0,1,0,1]

items = len(fejlesztes)

m = Model("fradir")

#decision variable
x = [m.add_var(var_type=BINARY) for i in range(items)]
fejlesztes_Dvar = [m.add_var(var_type=INTEGER, lb=1, ub=6) for i in range(items)]

#objective
m.objective = maximize(xsum(koltseg * fejlesztes[i] * x[i] for i in range(items)))

#cons
#m += xsum(y[i] * x[i] for i in range(items)) >= 2

#m += xsum(z[i] * x[i] for i in range(items)) >= 2

#m += xsum(((y[i] + z[i]) % 2) * x[i]  for i in range(items)) == 0

for i in range(items):
    m += 10 * x[i] <= fejlesztes[i]

#m += xsum(( fejlesztes[i] * x[i]  for i in range(items))) <= 14

m += xsum(x[i] for i in range(items)) == 2


m.optimize()

selected = [i for i in range(items) if x[i].x >= 0.99]
print("selected items: {}".format(selected))

selected items: [3, 6]


In [14]:
ff(3,4)

22

# GEKKO

In [6]:
import random
from gekko import GEKKO
import numpy as np

In [7]:
koltseg =  [11, 3,3, 6, 1, 9, 16,78]
fejlesztes = [17,67,4,61,16,42,156, 7]

y = [1,0,1,1,0,1,1,0]
z = [0,1,1,1,0,1,1,1]

prob = [0.5, 1.2, 1.5, 1.7, 0.1, 1.8, 0.4, 1.9]

items = len(koltseg)

# Create model
m = GEKKO()

# Variables
x = m.Array(m.Var,items,lb=0,ub=1,integer=True)
fejlesztes_Dvar = m.Array(m.Var, items, lb=1,ub=6, integer=True)
b = m.Array(m.Var, items, lb=1,ub=8, integer=True)
#x2 = m.Array(m.Var, len(w),lb=0,ub=1,integer = True)

# Objective
m.Maximize(sum(koltseg[i] * fejlesztes_Dvar[i] * x[i] for i in range(items) ))

# Constraint
m.Equation(sum([(y[i] + z[i]) * x[i] for i in range(items)]) == 2)

m.Equation(sum([x[i] for i in range(items)]) == 1)

m.Equation(sum((z[i] * prob[i]) *x[i] for i in range(items)) <=1.6 )

# Optimize with APOPT
m.options.SOLVER = 1

m.solve(disp = False)

# Print the value of the variables at the optimum
print(x)
for i in range(len(x)):
    if x[i][0] == 1.0:
        print("Sorszam: {}., koltseg: {}, fejlesztes: {}".format(i,koltseg[i], fejlesztes_Dvar[i][0]))

[[0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [1.0] [0.0]]
Sorszam: 6., koltseg: 16, fejlesztes: 6.0


In [2]:
data = np.load('italy.npy')

In [13]:
pd.DataFrame(data[0]).describe()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
count,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,...,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000
mean,1.154713,1.174508,1.196344,1.220365,1.246715,1.275495,1.306710,1.340423,1.376788,1.415944,...,3.194826,3.325022,3.459114,3.597036,3.738632,3.883800,4.032280,4.183553,4.337154,4.492752
std,0.578387,0.617518,0.658248,0.700550,0.744388,0.789728,0.836545,0.884773,0.934307,0.985046,...,2.112041,2.145883,2.176857,2.204735,2.229386,2.250615,2.268428,2.283199,2.295290,2.304934
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.260501,1.422601,1.584701,1.746801,1.908901,2.071001,2.233101,2.395201,2.557301,2.719401
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.691067,2.853167,3.015267,3.177367,3.339467,3.501567,3.663667,3.825767,3.987867,4.149967
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.004125,...,4.570325,4.732425,4.894525,5.056625,5.218725,5.380825,5.542925,5.705025,5.867125,6.029225
max,6.113586,6.275686,6.437786,6.599886,6.761986,6.924086,7.086186,7.248286,7.410386,7.572486,...,11.138686,11.300786,11.462886,11.624986,11.787086,11.949186,12.111286,12.273386,12.435486,12.597586
